In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ASM']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('airway smooth muscle', 824),
 ('acid sphingomyelinase', 247),
 ('acibenzolar s methyl', 24),
 ('allele specific methyl', 14),
 ('appendicular skeletal muscle mass', 10),
 ('artificial sputum medium', 9),
 ('smooth muscle area', 9),
 ('anesthesia sensitive memory', 8),
 ('airway smooth muscle cells', 8),
 ('allele specific dna methyl', 7),
 ('acid soluble metabolites', 5),
 ('asian summer monsoon', 5),
 ('acibenzolar s methyl treated', 4),
 ('acid soluble matrix', 3),
 ('alpine swamp meadow', 3),
 ('aggressive sm', 3),
 ('aggressive system mastocytosis', 3),
 ('atrial siphon muscle', 3),
 ('arterial smooth muscle', 3)]

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'acibenzolar s methyl': 'CHEBI:CHEBI:73178',
  'acibenzolar s methyl treated': 'ungrounded',
  'acid soluble matrix': 'ungrounded',
  'acid soluble metabolites': 'ungrounded',
  'acid sphingomyelinase': 'HGNC:11120',
  'aggressive sm': 'ungrounded',
  'aggressive system mastocytosis': 'ungrounded',
  'airway smooth muscle': 'BTO:0001660',
  'airway smooth muscle cells': 'MESH:D032389',
  'allele specific dna methyl': 'ungrounded',
  'allele specific methyl': 'ungrounded',
  'alpine swamp meadow': 'ungrounded',
  'anesthesia sensitive memory': 'ungrounded',
  'appendicular skeletal muscle mass': 'ungrounded',
  'arterial smooth muscle': 'ungrounded',
  'artificial sputum medium': 'ungrounded',
  'asian summer monsoon': 'ungrounded',
  'atrial siphon muscle': 'ungrounded',
  'smooth muscle area': 'ungrounded'},
 {'CHEBI:CHEBI:73178': 'acibenzolar-S-methyl',
  'HGNC:11120': 'SMPD1',
  'BTO:0001660': 'respiratory smooth muscle',
  'MESH:D032389': 'Myocytes, Smooth Muscle'},
 ['BTO:000166

In [7]:
grounding_map, names, pos_labels = [{'acibenzolar s methyl': 'CHEBI:CHEBI:73178',
  'acibenzolar s methyl treated': 'CHEBI:CHEBI:73178',
  'acid soluble matrix': 'ungrounded',
  'acid soluble metabolites': 'ungrounded',
  'acid sphingomyelinase': 'HGNC:11120',
  'aggressive sm': 'ungrounded',
  'aggressive system mastocytosis': 'ungrounded',
  'airway smooth muscle': 'BTO:0001660',
  'airway smooth muscle cells': 'BTO:0001660',
  'allele specific dna methyl': 'ungrounded',
  'allele specific methyl': 'ungrounded',
  'alpine swamp meadow': 'ungrounded',
  'anesthesia sensitive memory': 'ungrounded',
  'appendicular skeletal muscle mass': 'ungrounded',
  'arterial smooth muscle': 'ungrounded',
  'artificial sputum medium': 'ungrounded',
  'asian summer monsoon': 'ungrounded',
  'atrial siphon muscle': 'ungrounded',
  'smooth muscle area': 'ungrounded'},
 {'CHEBI:CHEBI:73178': 'acibenzolar-S-methyl',
  'HGNC:11120': 'SMPD1',
  'BTO:0001660': 'respiratory smooth muscle'},
 ['BTO:0001660', 'CHEBI:CHEBI:73178', 'HGNC:11120']]

In [8]:
excluded_longforms = []

In [9]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [10]:
additional_entities = {}

In [11]:
unambiguous_agent_texts = {}

In [12]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [13]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [14]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [15]:
intersection1

[]

In [16]:
intersection2

[]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-26 01:58:53] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-26 01:59:06] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9821676718486613 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'BTO:0001660': 609,
  'HGNC:11120': 185,
  'CHEBI:CHEBI:73178': 19,
  'ungrounded': 51},
 'f1': {'mean': 0.982168, 'std': 0.010477},
 'precision': {'mean': 0.979454, 'std': 0.008262},
 'recall': {'mean': 0.986488, 'std': 0.012511},
 'ungrounded': {'f1': {'mean': 0.916725, 'std': 0.025347},
  'pr': {'mean': 0.863636, 'std': 0.044906},
  'rc': {'mean': 0.98, 'std': 0.04}},
 'CHEBI:CHEBI:73178': {'f1': {'mean': 0.847619, 'std': 0.106053},
  'pr': {'mean': 0.75, 'std': 0.158114},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:11120': {'f1': {'mean': 0.980816, 'std': 0.018918},
  'pr': {'mean': 0.978378, 'std': 0.031519},
  'rc': {'mean': 0.983759, 'std': 0.013286}},
 'BTO:0001660': {'f1': {'mean': 0.987005, 'std': 0.007836},
  'pr': {'mean': 0.996721, 'std': 0.006557},
  'rc': {'mean': 0.977484, 'std': 0.009283}}}

In [18]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [19]:
disamb.dump(model_name, results_path)

In [20]:
print(disamb.info())

Disambiguation model for ASM

Produces the disambiguations:
	SMPD1*	HGNC:11120
	acibenzolar-S-methyl*	CHEBI:CHEBI:73178
	respiratory smooth muscle*	BTO:0001660

Class level metrics:
--------------------
Grounding                	Count	F1     
respiratory smooth muscle*	609	  0.987
                    SMPD1*	185	0.98082
               Ungrounded	 51	0.91672
     acibenzolar-S-methyl*	 19	0.84762

Weighted Metrics:
-----------------
	F1 score:	0.98217
	Precision:	0.97945
	Recall:		0.98649

* Positive labels
See Docstring for explanation



In [21]:
model_to_s3(disamb)